# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm

# Challenge 1 - Import and Describe the Dataset

In this lab, we will use a dataset containing information about customer preferences. We will look at how much each customer spends in a year on each subcategory in the grocery store and try to find similarities using clustering.

The dataset is located [here](https://drive.google.com/file/d/1z1gYSD32ktbHuKSzB5JVS_u4YsLibh5F/view?usp=sharing), please download it and place it in the data folder.

In [ ]:
# loading the data:

customer = pd.read_csv('../data/Wholesale_customer_ data.csv')
 
customer.head()

#### Explore the dataset with mathematical and visualization techniques. What do you find?

Checklist:

* What does each column mean?
* Any categorical data to convert?
* Any missing data to remove?
* Column collinearity - any high correlations?
* Descriptive statistics - any outliers to remove?
* Column-wise data distribution - is the distribution skewed?
* Etc.

Additional info: Over a century ago, an Italian economist named Vilfredo Pareto discovered that roughly 20% of the customers account for 80% of the typical retail sales. This is called the [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle). Check if this dataset displays this characteristic.

In [ ]:
customer.dtypes

# All columns have interger values and there are no object values

In [ ]:
customer.isna().sum()
# There are no missing values in the columns

In [ ]:
customer.corr()

In [ ]:
customer.describe()

In [ ]:
# Removing outliers from the dataframe
# This should remove the entire row of the dataframe wherever there is an outlier
# Reference: https://kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python

from scipy.stats import zscore

# Calculate z-score of the dataframe
z_scores = stats.zscore(customer)

# Calculate the absolute value element-wise.
# np.abs is a shorthand for this function.
# Call numpy.abs(x) with x as the previous result to convert each element in x to its absolute value.

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
customer = customer[filtered_entries]

customer



In [ ]:
# Calculating Skew
# Skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean. 

customer.skew(axis = 0) 


In [ ]:
# OBSERVATIONS:

# 1.What does each column mean?
# A. Each column indicates the type of item the customer is buying


# 2.Any categorical data to convert?
# A.There is no categorical data to convert

# 3.Any missing data to remove?
# A.There are no missing values in the df


# 4.Column collinearity - any high correlations?
# A.There is no high collinearity between the different columns

# 5.Descriptive statistics - any outliers to remove?
# A.There seemed to be quite a few outliers. I removed all the outliers.


# 6.Column-wise data distribution - is the distribution skewed?
# A. All columns had a positive skew except Column "Region" which had a negative skew.

# Challenge 2 - Data Cleaning and Transformation

If your conclusion from the previous challenge is the data need cleaning/transformation, do it in the cells below. However, if your conclusion is the data need not be cleaned or transformed, feel free to skip this challenge. But if you do choose the latter, please provide rationale.

In [ ]:
# I already cleaned the data by removing all all the outliers.
# So, I dont have to perform any additional cleaning/transformation.

# Challenge 3 - Data Preprocessing

One problem with the dataset is the value ranges are remarkably different across various categories (e.g. `Fresh` and `Grocery` compared to `Detergents_Paper` and `Delicassen`). If you made this observation in the first challenge, you've done a great job! This means you not only completed the bonus questions in the previous Supervised Learning lab but also researched deep into [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). Keep on the good work!

Diverse value ranges in different features could cause issues in our clustering. The way to reduce the problem is through feature scaling. We'll use this technique again with this dataset.

#### We will use the `StandardScaler` from `sklearn.preprocessing` and scale our data. Read more about `StandardScaler` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*After scaling your data, assign the transformed data to a new variable `customers_scale`.*

In [ ]:
# Your import here:

from sklearn.preprocessing import StandardScaler

# Your code here:

scaler = StandardScaler()
scaled_customer = scaler.fit_transform(customer)
scaled_customer = pd.DataFrame(scaled_customer)
scaled_customer

# Challenge 3 - Data Clustering with K-Means

Now let's cluster the data with K-Means first. Initiate the K-Means model, then fit your scaled data. In the data returned from the `.fit` method, there is an attribute called `labels_` which is the cluster number assigned to each data record. What you can do is to assign these labels back to `customers` in a new column called `customers['labels']`. Then you'll see the cluster results of the original data.

In [ ]:
# Your code here:
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

# n_clusters: The number of clusters to form as well as the number of centroids to generate.
# By default n_clusters=8

from sklearn.cluster import KMeans

customer_kmeans = KMeans().fit(scaled_customer)
customer_kmeans.labels_


In [ ]:
customer['labels'] = customer_kmeans.labels_

customer

Count the values in `labels`.

In [ ]:
# Your code here:

customer['labels'].value_counts() 

# Challenge 4 - Data Clustering with DBSCAN

Now let's cluster the data using DBSCAN. Use `DBSCAN(eps=0.5)` to initiate the model, then fit your scaled data. In the data returned from the `.fit` method, assign the `labels_` back to `customers['labels_DBSCAN']`. Now your original data have two labels, one from K-Means and the other from DBSCAN.

In [ ]:
# Your code here
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
# eps default=0.5

from sklearn.cluster import DBSCAN

customer_clustering = DBSCAN(eps=0.5).fit(scaled_customer)

customer_clustering.labels_ 


In [ ]:
customer['labels_DBSCAN'] = customer_clustering.labels_ 

customer


Count the values in `labels_DBSCAN`.

In [ ]:
# Your code here

customer['labels_DBSCAN'] .value_counts() 

# Challenge 5 - Compare K-Means with DBSCAN

Now we want to visually compare how K-Means and DBSCAN have clustered our data. We will create scatter plots for several columns. For each of the following column pairs, plot a scatter plot using `labels` and another using `labels_DBSCAN`. Put them side by side to compare. Which clustering algorithm makes better sense?

Columns to visualize:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualize `Detergents_Paper` as X and `Milk` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:

# Create a figure of a fixed size and axes
fig, [ax1, ax2] = plt.subplots(1,2, figsize = (19,6))


# K-Means Labels Scatter Plot
ax1.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
ax1.set_title('Distribution of K-Means Labels')
ax1.set_xlabel('Detergents_Paper')
ax1.set_ylabel('Milk')

#DBSCAN Labels Scatter Plot
ax2.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels_DBSCAN"])
ax2.set_title('Distribution of DBSCAN Labels')
ax2.set_xlabel('Detergents_Paper')
ax2.set_ylabel('Milk')


plt.show()



Visualize `Grocery` as X and `Fresh` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:

# Create a figure of a fixed size and axes
fig, [ax1, ax2] = plt.subplots(1,2, figsize = (19,6))


# K-Means Labels Scatter Plot
ax1.scatter(x=customer["Grocery"], y=customer["Fresh"], c=customer["labels"])
ax1.set_title('Distribution of K-Means Labels')
ax1.set_xlabel('Detergents_Paper')
ax1.set_ylabel('Milk')

#DBSCAN Labels Scatter Plot
ax2.scatter(x=customer["Grocery"], y=customer["Fresh"], c=customer["labels_DBSCAN"])
ax2.set_title('Distribution of DBSCAN Labels')
ax2.set_xlabel('Detergents_Paper')
ax2.set_ylabel('Milk')


plt.show()



Visualize `Frozen` as X and `Delicassen` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:

# Create a figure of a fixed size and axes
fig, [ax1, ax2] = plt.subplots(1,2, figsize = (19,6))


# K-Means Labels Scatter Plot
ax1.scatter(x=customer["Frozen"], y=customer["Delicassen"], c=customer["labels"])
ax1.set_title('Distribution of K-Means Labels')
ax1.set_xlabel('Detergents_Paper')
ax1.set_ylabel('Milk')

#DBSCAN Labels Scatter Plot
ax2.scatter(x=customer["Frozen"], y=customer["Delicassen"], c=customer["labels_DBSCAN"])
ax2.set_title('Distribution of DBSCAN Labels')
ax2.set_xlabel('Detergents_Paper')
ax2.set_ylabel('Milk')


plt.show()

Let's use a groupby to see how the mean differs between the groups. Group `customers` by `labels` and `labels_DBSCAN` respectively and compute the means for all columns.

In [ ]:
# Your code here:

customer.groupby(['labels','labels_DBSCAN']).mean()


In [ ]:
customer.groupby('labels').mean()

In [ ]:
customer.groupby('labels_DBSCAN').mean()

Which algorithm appears to perform better?

In [ ]:
# Your observations here

# I think for this customer data, K-means looks like a better algorithm. 
# Since there is a wide variety of data, it may not be that accurate to have DBSCAN as it clustered into 2 lables.

# Bonus Challenge 2 - Changing K-Means Number of Clusters

As we mentioned earlier, we don't need to worry about the number of clusters with DBSCAN because it automatically decides that based on the parameters we send to it. But with K-Means, we have to supply the `n_clusters` param (if you don't supply `n_clusters`, the algorithm will use `8` by default). You need to know that the optimal number of clusters differs case by case based on the dataset. K-Means can perform badly if the wrong number of clusters is used.

In advanced machine learning, data scientists try different numbers of clusters and evaluate the results with statistical measures (read [here](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). We are not using statistical measures today but we'll use our eyes instead. In the cells below, experiment with different number of clusters and visualize with scatter plots. What number of clusters seems to work best for K-Means?

In [ ]:
# Your code here

from sklearn.cluster import KMeans

# KMeans for n_clusters=2
customer_kmeans = KMeans(n_clusters=2).fit(scaled_customer)
customer['labels'] = customer_kmeans.labels_

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# KMeans for n_clusters=4
customer_kmeans = KMeans(n_clusters=4).fit(scaled_customer)
customer['labels'] = customer_kmeans.labels_

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# KMeans for n_clusters=6
customer_kmeans = KMeans(n_clusters=6).fit(scaled_customer)
customer['labels'] = customer_kmeans.labels_

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# Your comment here

# From the above 3 plots with n_clusters values 2, 4 and 6, it looks like the lower the number of clusters the better it looks.
# I am not fully sure though.

# Bonus Challenge 3 - Changing DBSCAN `eps` and `min_samples`

Experiment changing the `eps` and `min_samples` params for DBSCAN. See how the results differ with scatter plot visualization.

In [ ]:
# Your code here
# eps of 0.5 is the default


# DBSCAN for eps = 0.2

from sklearn.cluster import DBSCAN

customer_clustering = DBSCAN(eps=0.2).fit(scaled_customer)

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# DBSCAN for eps = 0.6

from sklearn.cluster import DBSCAN

customer_clustering = DBSCAN(eps=0.6).fit(scaled_customer)

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# DBSCAN for eps = 0.8

from sklearn.cluster import DBSCAN

customer_clustering = DBSCAN(eps=0.8).fit(scaled_customer)

plt.scatter(x=customer["Detergents_Paper"], y=customer["Milk"], c=customer["labels"])
plt.show()

In [ ]:
# Your comment here

# From the above 3 plots with eps values 0.2, 0.6 and 0.8:
# I find it very hard to say which one is better just based on the visualization